In [7]:
import pandas as pd
#数据源:https://grouplens.org/datasets/movielens/；1M Movielens数据集中的ratings.csv,movies.csv,tags.csv
file_1 = pd.read_csv('./learning/L7/movielens/ml-latest-small/movies.csv')
file_2 = pd.read_csv('./learning/L7/movielens/ml-latest-small/ratings.csv')
file_3 = pd.read_csv('./learning/L7/movielens/ml-latest-small/tags.csv')
#合并3个文件生成DeepFM的数据集
file_3 = file_3.drop('timestamp' , axis =1) 
file_4 = pd.merge(file_2 , file_3)
file_5 = pd.merge(file_4 , file_1)
file_5.to_csv('./learning/L7/movielens/ml-latest-small/action2_movielens.csv' , index = 0)
print('Done')

Done


In [10]:
print(file_5.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3476 entries, 0 to 3475
Data columns (total 7 columns):
userId       3476 non-null int64
movieId      3476 non-null int64
rating       3476 non-null float64
timestamp    3476 non-null int64
tag          3476 non-null object
title        3476 non-null object
genres       3476 non-null object
dtypes: float64(1), int64(3), object(3)
memory usage: 217.2+ KB
None


In [21]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM
from deepctr.inputs import SparseFeat,get_feature_names

#数据加载
data = pd.read_csv('./learning/L7/movielens/ml-latest-small/action2_movielens.csv' )
sparse_features = ['userId', 'movieId', 'timestamp', 'tag', 'title', 'genres']
target = ['rating']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
print(fixlen_feature_columns)
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=10, verbose=True, validation_split=0.2, )
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

[SparseFeat(name='userId', vocabulary_size=54, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='userId', group_name='default_group'), SparseFeat(name='movieId', vocabulary_size=1464, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='movieId', group_name='default_group'), SparseFeat(name='timestamp', vocabulary_size=1587, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='timestamp', group_name='default_group'), SparseFeat(name='tag', vocabulary_size=1543, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='tag', group_name='default_group'), SparseFeat(name='title', vocabulary_size=1464, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='title', group_name='default_group'), SparseFeat(name='genres', vocabulary_size=364, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='genres', group_name='default_group')]
Train on 2224 samples, validate on 556 samples
Epoch 1/10


C:\Users\TAOXUEJIE-PSD\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\TAOXUEJIE-PSD\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


2224/2224 [==============================] - 2s 731us/sample - loss: 16.3121 - mse: 16.3121 - val_loss: 15.5509 - val_mse: 15.5509
Epoch 2/10
2224/2224 [==============================] - 0s 24us/sample - loss: 14.4854 - mse: 14.4854 - val_loss: 13.1709 - val_mse: 13.1709
Epoch 3/10
2224/2224 [==============================] - 0s 21us/sample - loss: 11.3598 - mse: 11.3598 - val_loss: 9.2413 - val_mse: 9.2413
Epoch 4/10
2224/2224 [==============================] - 0s 23us/sample - loss: 6.5916 - mse: 6.5915 - val_loss: 4.0040 - val_mse: 4.0039
Epoch 5/10
2224/2224 [==============================] - 0s 24us/sample - loss: 1.7573 - mse: 1.7572 - val_loss: 0.6831 - val_mse: 0.6830
Epoch 6/10
2224/2224 [==============================] - 0s 21us/sample - loss: 0.8660 - mse: 0.8659 - val_loss: 0.7586 - val_mse: 0.7585
Epoch 7/10
2224/2224 [==============================] - 0s 19us/sample - loss: 0.5629 - mse: 0.5628 - val_loss: 0.5555 - val_mse: 0.5554
Epoch 8/10
2224/2224 [===================